### Basic Concept:
KNN operates under the assumption that similar data points should have similar labels (in classification) or similar values (in regression). It's a lazy learning algorithm because it does not explicitly learn a model during training phase. Instead, it memorizes the training dataset and makes predictions based on new data points' proximity to known data points.

### Algorithm Steps:
1. **Choose K**: Determine the number \( K \) of neighbors to consider.
2. **Calculate Distance**: Compute the distance (typically Euclidean distance) between the new data point and all points in the training dataset.
3. **Select K Neighbors**: Identify the K nearest neighbors based on the computed distances.
4. **Majority Vote (Classification)** / **Average (Regression)**: For classification, assign the class label by majority vote among the K neighbors. For regression, predict the average of the target values of the K neighbors.

### Key Parameters:
- **K**: Number of neighbors to consider.
- **Distance Metric**: Typically Euclidean distance, but other metrics like Manhattan distance or cosine similarity can also be used depending on the data and problem.

### Advantages:
- **Simple**: Easy to understand and implement.
- **No Training Time**: No explicit training phase; predictions are made directly based on the training data.
- **Versatile**: Can be used for both classification and regression tasks.
- **Non-parametric**: Can handle complex decision boundaries.

### Disadvantages:
- **Computational Cost**: As the size of the training dataset grows, the prediction time can be slow because it requires computing distances to all training samples.
- **Storage**: Needs to store the entire training dataset.
- **Sensitive to Outliers**: Outliers can significantly affect the prediction, especially in small \( K \).

### Practical Considerations:
- **Choosing K**: Typically chosen using techniques like cross-validation to find an optimal balance between bias and variance, or square root of the number of data.
- **Normalization**: Data normalization (scaling features) can be crucial because KNN is sensitive to the scale of features.
- **Distance Metrics**: Selection of appropriate distance metrics can impact the algorithm’s performance.

### Applications:
- **Classification**: Identifying the class of a new observation based on labeled examples.
- **Regression**: Predicting a continuous-valued attribute based on the average of its neighbors.
- **Anomaly Detection**: Identifying unusual data points based on their distance to their nearest neighbors.

In [ ]:
from sklearn import datasets
import math
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Load the wine dataset
X, y = datasets.load_wine(as_frame=True, return_X_y=True)

# Standardize the data
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Apply PCA to reduce to 3 components
pca = PCA(n_components=3)
X = pca.fit_transform(X)

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=42)

# Determine the value of k
k = math.floor(math.sqrt(len(y_test)))
k = k - 1 if (k % 2 == 0) else k

# Priority Queue Node class
class PriorityQueueNode:
    def __init__(self, value, pr):
        self.data = value
        self.priority = pr
        self.next = None

# Priority Queue class
class PriorityQueue:
    def __init__(self):
        self.front = None

    def isEmpty(self):
        return self.front is None

    def push(self, value, priority):
        newNode = PriorityQueueNode(value, priority)
        if self.isEmpty() or self.front.priority > priority:
            newNode.next = self.front
            self.front = newNode
        else:
            temp = self.front
            while temp.next and temp.next.priority <= priority:
                temp = temp.next
            newNode.next = temp.next
            temp.next = newNode

    def pop(self):
        if self.isEmpty():
            return None
        else:
            highest_priority_node = self.front
            self.front = self.front.next
            return highest_priority_node.data

# KNN Classifier class
class KNNClassifier:
    def __init__(self, k):
        self.k = k

    def calc_dist(self, p1, p2):
        return math.sqrt(sum((p1[i] - p2[i])**2 for i in range(len(p1))))

    def get_highest_vote(self, predicted_labels):
        labels_count = [0] * 3
        for label in predicted_labels:
            labels_count[label] += 1
        return labels_count.index(max(labels_count))

    def get_knn_labels(self, k, X_train, y_train, X_point):
        labels = PriorityQueue()
        for i in range(len(X_train)):
            distance = self.calc_dist(X_train[i], X_point)
            labels.push(y_train.iloc[i], distance)
        knn_labels = [labels.pop() for _ in range(k)]
        return knn_labels

    def predict(self, X_train, y_train, X_point):
        neighbor_labels = self.get_knn_labels(self.k, X_train, y_train, X_point)
        return self.get_highest_vote(neighbor_labels)

# Function to calculate accuracy
def accuracy(y_test, predicted_labels):
    correct = sum(pred == true for pred, true in zip(predicted_labels, y_test))
    return correct / len(y_test)

# Create the KNN classifier
knn_clf = KNNClassifier(k)

# Predict labels for the test set
predicted_labels = [knn_clf.predict(X_train, y_train, X_point) for X_point in X_test]

# Print the accuracy
print("Accuracy:", accuracy(y_test, predicted_labels))


Accuracy: 0.9629629629629629
